# 문제 2: 치와와 vs 머핀 분류

- **데이터셋**: Kaggle Muffin vs Chihuahua (128×128 RGB)
- **검증**: StratifiedKFold 5겹 교차검증
- **평가**: Accuracy, F1 Score (Micro), F1 Score (Macro)

## 모델 구조
1. **LeNet-5** (베이스라인) - 수업시간에 배운 고전적 CNN
2. **VGGNet-style** - 작은 3×3 커널을 깊게 쌓은 구조
3. **ResNetCNN** - Residual Connection으로 깊은 학습
4. **EfficientNet-Lite** - 간소화된 MBConv + SE 구조
5. **EfficientNet-B0** (최고 성능 예상) - 원본 B0 아키텍처

## 평가 척도 설명
- **Accuracy**: 전체 정확도
- **F1 (Micro)**: 전체 TP/FP/FN 합산 후 계산 (= Accuracy, 균형 데이터에서)
- **F1 (Macro)**: 클래스별 F1의 평균 (불균형 데이터에서 유용)

치와와와 머핀은 외형이 매우 유사하여 세밀한 특징 추출이 필요

## 1. 라이브러리

In [ ]:
# -------------------------------
# 공용 라이브러리 및 유틸 불러오기
# - torch/torchvision: 모델 정의, 변환, Mixed Precision
# - sklearn: 교차검증 및 지표 계산
# - PIL: 이미지 로딩
# - matplotlib: 시각화 및 한글 폰트 설정
# - zipfile/os: Kaggle 데이터 압축 해제 및 경로 관리
# -------------------------------
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.amp import autocast, GradScaler  # Mixed Precision (새 API)
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os
import zipfile

# 한글 폰트 설정 (Linux 서버 환경) - 그래프 라벨 깨짐 방지
def set_korean_font():
    """시스템에서 사용 가능한 한글 폰트를 자동 설정"""
    font_candidates = [
        'NanumGothic', 'NanumBarunGothic', 'Malgun Gothic',
        'AppleGothic', 'DejaVu Sans', 'Noto Sans CJK KR'
    ]
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    
    for font in font_candidates:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            print(f"한글 폰트 설정: {font}")
            return
    
    print("한글 폰트를 찾지 못했습니다. 영문 라벨을 사용합니다.")

set_korean_font()

# GPU 우선, Mac이면 MPS, 그 외 CPU 선택
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Device: {device}")

# GPU 정보 출력 (리포트용)
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")



In [ ]:
# --------------------------------------------
# H100 GPU 최적화 기본 설정 + 시드 고정 헬퍼
# --------------------------------------------
BATCH_SIZE = 128          # H100: 80GB 메모리, 배치 크기 증가
NUM_WORKERS = 8           # 데이터 로딩 병렬화
PIN_MEMORY = True         # GPU 메모리 전송 최적화
USE_AMP = True            # Mixed Precision (BF16/FP16)

def set_seed(seed):
    """torch/np 시드를 모두 고정하여 StratifiedKFold 일관성 유지"""
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

# cuDNN 최적화 (고정 입력 크기에서 성능 향상)
torch.backends.cudnn.benchmark = True

print(f"H100 최적화 설정:")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Num Workers: {NUM_WORKERS}")
print(f"  - Pin Memory: {PIN_MEMORY}")
print(f"  - Mixed Precision (AMP): {USE_AMP}")
print(f"  - cuDNN Benchmark: {torch.backends.cudnn.benchmark}")



## 2. 데이터셋 다운로드

Kaggle API 필요: `pip install kaggle`

In [ ]:
# --------------------------------------------
# Kaggle 데이터 다운로드 디렉토리 준비
# --------------------------------------------
DATA_DIR = './data/chihuahua_muffin'
os.makedirs(DATA_DIR, exist_ok=True)



In [ ]:
# --------------------------------------------
# Kaggle 데이터셋 다운로드 (치와와 vs 머핀)
# --------------------------------------------
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification -p {DATA_DIR}



In [ ]:
# --------------------------------------------
# 다운로드한 zip 압축 해제
# --------------------------------------------
zip_path = os.path.join(DATA_DIR, 'muffin-vs-chihuahua-image-classification.zip')
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR)
print("압축 해제 완료")



## 3. 데이터 준비

In [ ]:
# --------------------------------------------
# Custom Dataset 정의
# - 폴더 구조: chihuahua/, muffin/
# - 이미지 경로와 라벨을 미리 스캔하여 리스트로 보관
# --------------------------------------------
class ChihuahuaMuffinDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for class_name, class_idx in [('chihuahua', 0), ('muffin', 1)]:
            class_dir = os.path.join(data_dir, class_name)
            if os.path.exists(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.endswith(('.jpg', '.jpeg', '.png')):
                        self.image_paths.append(os.path.join(class_dir, img_name))
                        self.labels.append(class_idx)
        
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # PIL로 로드 후 RGB 변환, transform 적용
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]



In [ ]:
# --------------------------------------------
# 데이터 변환 정의
# - train: 색상/밝기 증강 포함 (클래스 유사도 대응)
# - test: 증강 없이 정규화만 적용
# --------------------------------------------
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



In [ ]:
# --------------------------------------------
# 데이터셋 로드 (train/test 분리 제공)
# --------------------------------------------
train_dir = os.path.join(DATA_DIR, 'train')
test_dir = os.path.join(DATA_DIR, 'test')

train_dataset = ChihuahuaMuffinDataset(train_dir)
test_dataset = ChihuahuaMuffinDataset(test_dir, transform=test_transform)

print(f"훈련: {len(train_dataset)}, 테스트: {len(test_dataset)}")
print(f"클래스 분포: chihuahua={sum(train_dataset.labels==0)}, muffin={sum(train_dataset.labels==1)}")



In [ ]:
# --------------------------------------------
# 샘플 시각화 - 치와와 vs 머핀의 유사성 확인
# --------------------------------------------
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
temp_ds = ChihuahuaMuffinDataset(train_dir, transform=test_transform)

chi_idx = np.where(temp_ds.labels == 0)[0][:5]
muf_idx = np.where(temp_ds.labels == 1)[0][:5]

for i, idx in enumerate(chi_idx):
    img, _ = temp_ds[idx]
    img = img * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    axes[0, i].imshow(img.permute(1,2,0).clip(0,1))
    axes[0, i].set_title('Chihuahua')
    axes[0, i].axis('off')

for i, idx in enumerate(muf_idx):
    img, _ = temp_ds[idx]
    img = img * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    axes[1, i].imshow(img.permute(1,2,0).clip(0,1))
    axes[1, i].set_title('Muffin')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()



## 4. 모델 정의

### 망 구조 표기법
- **C**: Convolution (합성곱) - 공간적 특징 추출
- **DWC**: Depthwise Separable Conv - 채널별 독립 합성곱
- **BN**: Batch Normalization - 학습 안정화
- **R**: ReLU / **T**: Tanh / **Swish**: x·σ(x) - 활성화 함수
- **AvgP** / **MaxP**: Average/Max Pooling
- **GAP**: Global Average Pooling
- **D**: Dropout - 정규화
- **FC**: Fully Connected
- **Res**: Residual Connection
- **SE**: Squeeze-and-Excitation - 채널 attention
- **MBConv**: Mobile Inverted Bottleneck Conv

---

### 4.1 LeNet-5 (베이스라인, LeCun et al., 1998)

**구조**: `C(3,6,5×5) → T → AvgP → C(6,16,5×5) → T → AvgP → C(16,32,5×5) → T → AvgP → Flat → FC → T → FC → T → FC`

**특징**:
- 최초의 성공적인 CNN (수업시간에 배운 모델)
- 5×5 큰 커널로 특징 추출
- Tanh 활성화 함수 사용

**효과/한계**:
- ✅ CNN의 기본 원리 적용
- ❌ 얕은 구조로 세밀한 특징 추출 한계
- ❌ 치와와 눈과 머핀 초콜릿칩 구분 어려움

In [ ]:
class LeNet5(nn.Module):
    """
    LeNet-5 (LeCun et al., 1998) - 128x128 RGB 버전
    구조: C(5x5) → T → AvgP → C(5x5) → T → AvgP → C(5x5) → T → AvgP → Flat → FC → T → FC → T → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        # 레이어를 ModuleList에 저장해 구조를 한눈에 파악
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 6, 5),                # C(3,6,5x5): 128 → 124
            nn.Tanh(),                          # T
            nn.AvgPool2d(2, 2),                 # AvgP: 124 → 62
            nn.Conv2d(6, 16, 5),                # C(6,16,5x5): 62 → 58
            nn.Tanh(),                          # T
            nn.AvgPool2d(2, 2),                 # AvgP: 58 → 29
            nn.Conv2d(16, 32, 5),               # C(16,32,5x5): 29 → 25
            nn.Tanh(),                          # T
            nn.AvgPool2d(2, 2),                 # AvgP: 25 → 12
            nn.Flatten(),                       # Flat: (B,32,12,12) → (B,4608)
            nn.Linear(32 * 12 * 12, 120),       # FC
            nn.Tanh(),                          # T
            nn.Linear(120, 84),                 # FC
            nn.Tanh(),                          # T
            nn.Linear(84, num_classes)          # FC: 출력
        ])
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

print("LeNet-5: C(5x5) → T → AvgP → C(5x5) → T → AvgP → C(5x5) → T → AvgP → Flat → FC×3")
print(LeNet5())
print(f"파라미터: {sum(p.numel() for p in LeNet5().parameters()):,}")



### 4.2 VGGNet-style (Simonyan & Zisserman, 2014)

**구조**: `[C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×3 → MaxP → [C(3×3) → BN → R]×3 → GAP → D → FC`

**특징**:
- 작은 3×3 커널을 깊게 쌓음 (VGG의 핵심)
- 3×3 두 번 = 5×5 수용 영역, 더 적은 파라미터
- BN + ReLU로 현대화

**효과**:
- ✅ 깊은 네트워크로 세밀한 패턴 학습
- ✅ 작은 커널로 파라미터 효율성
- ⚠️ Skip connection 없어 gradient 문제 가능

In [ ]:
class VGGBlock(nn.Module):
    """VGG Block: [C(3x3) → BN → R] × n_convs"""
    def __init__(self, in_ch, out_ch, n_convs=2):
        super().__init__()
        layers = []
        for i in range(n_convs):
            layers.extend([
                nn.Conv2d(in_ch if i == 0 else out_ch, out_ch, 3, 1, 1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU()
            ])
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


class VGGNet(nn.Module):
    """
    VGGNet-style - 128x128 RGB
    구조: VGGBlock×4 + MaxPool×4 + GAP + FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            VGGBlock(3, 32, n_convs=2),         # Block1: 128→64
            nn.MaxPool2d(2, 2),
            VGGBlock(32, 64, n_convs=2),        # Block2: 64→32
            nn.MaxPool2d(2, 2),
            VGGBlock(64, 128, n_convs=3),       # Block3: 32→16
            nn.MaxPool2d(2, 2),
            VGGBlock(128, 256, n_convs=3),      # Block4: 16→8
            nn.MaxPool2d(2, 2),
            nn.AdaptiveAvgPool2d(1),            # GAP
            nn.Dropout(0.5)
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("VGGNet: [C(3x3)→BN→R]×n → MaxP (4 blocks) → GAP → D → FC")
print(VGGNet())
print(f"파라미터: {sum(p.numel() for p in VGGNet().parameters()):,}")



### 4.3 ResNetCNN (He et al., 2015)

**구조**: `C(3,32,7×7,s2) → BN → R → MaxP → Res(32,64) → Res(64,128) → Res(128,256) → GAP → D(0.5) → FC(256,2)`

**ResBlock 내부**: `C(3×3) → BN → R → C(3×3) → BN → (+shortcut) → R`

**특징**:
- Residual Connection (skip connection)
- H(x) = F(x) + x로 잔차 학습

**효과**:
- ✅ Skip connection으로 gradient flow 개선
- ✅ Gradient vanishing 문제 해결
- ✅ VGGNet보다 깊게 학습 가능

In [ ]:
class ResidualBlock(nn.Module):
    """Residual Block: C → BN → R → C → BN → (+shortcut) → R"""
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_ch)
        ])
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        out += self.shortcut(x)
        return nn.ReLU()(out)


class ResNetCNN(nn.Module):
    """
    ResNetCNN - 128x128 RGB
    구조: C(7x7) → BN → R → MaxP → Res×3 → GAP → D → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 32, 7, 2, 3, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1),
            ResidualBlock(32, 64, 1),
            ResidualBlock(64, 128, 2),
            ResidualBlock(128, 256, 2),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.5)
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("ResNetCNN: C(7x7) → BN → R → MaxP → Res×3 → GAP → D → FC")
print(ResNetCNN())
print(f"파라미터: {sum(p.numel() for p in ResNetCNN().parameters()):,}")



### 4.4 EfficientNet-Lite (간소화된 EfficientNet)

**구조**: `C(3,32,3×3,s2) → BN → Swish → [MBConv]×8 → GAP → D → FC`

**MBConv (Mobile Inverted Bottleneck Conv)**:
`C(1×1,expand) → BN → Swish → DWC(3×3) → BN → Swish → SE → C(1×1,project) → BN → (+shortcut)`

**특징**:
- EfficientNet의 핵심 구성요소만 사용한 간소화 버전
- MBConv + SE + Swish 조합
- 128×128 입력에 맞게 채널 수 축소

**효과**:
- ✅ 파라미터 효율적 (원본 B0보다 가벼움)
- ✅ 빠른 학습 가능
- ⚠️ 원본 B0 대비 표현력 제한

In [ ]:
class Swish(nn.Module):
    """Swish 활성화: x * sigmoid(x), ReLU보다 smooth"""
    def forward(self, x):
        return x * torch.sigmoid(x)


class SEBlock(nn.Module):
    """Squeeze-and-Excitation Block - 채널 attention"""
    def __init__(self, channels, reduction=4):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(channels, channels // reduction),
            Swish(),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        ])
    
    def forward(self, x):
        b, c, _, _ = x.size()
        scale = x
        for layer in self.layers:
            scale = layer(scale)
        return x * scale.view(b, c, 1, 1)


class MBConv(nn.Module):
    """
    Mobile Inverted Bottleneck Conv (EfficientNet의 핵심)
    구조: C(1x1,expand) → BN → Swish → DWC(kxk) → BN → Swish → SE → C(1x1,project) → BN → (+shortcut)
    
    Args:
        in_ch: 입력 채널
        out_ch: 출력 채널
        kernel_size: Depthwise Conv 커널 크기 (3 or 5)
        expand_ratio: 채널 확장 비율
        stride: Depthwise Conv stride
        use_se: SE Block 사용 여부
    """
    def __init__(self, in_ch, out_ch, kernel_size=3, expand_ratio=4, stride=1, use_se=True):
        super().__init__()
        hidden_ch = in_ch * expand_ratio
        self.use_residual = (stride == 1 and in_ch == out_ch)
        padding = (kernel_size - 1) // 2
        
        layers = []
        # Expand 단계: 채널 확장
        if expand_ratio != 1:
            layers.extend([
                nn.Conv2d(in_ch, hidden_ch, 1, bias=False),
                nn.BatchNorm2d(hidden_ch),
                Swish()
            ])
        else:
            hidden_ch = in_ch
        
        # Depthwise Conv: 채널별 독립 필터
        layers.extend([
            nn.Conv2d(hidden_ch, hidden_ch, kernel_size, stride, padding, groups=hidden_ch, bias=False),
            nn.BatchNorm2d(hidden_ch),
            Swish()
        ])
        
        # SE Block (선택)
        if use_se:
            layers.append(SEBlock(hidden_ch))
        
        # Project 단계: 출력 채널로 축소
        layers.extend([
            nn.Conv2d(hidden_ch, out_ch, 1, bias=False),
            nn.BatchNorm2d(out_ch)
        ])
        
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        if self.use_residual:
            out = out + x
        return out


class EfficientNetLite(nn.Module):
    """
    EfficientNet-Lite - 간소화된 EfficientNet (128x128 RGB)
    구조: C(3x3,s2) → BN → Swish → MBConv×8 → GAP → D → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            Swish(),
            MBConv(32, 32, kernel_size=3, expand_ratio=1, stride=1),
            MBConv(32, 48, kernel_size=3, expand_ratio=4, stride=2),
            MBConv(48, 48, kernel_size=3, expand_ratio=4, stride=1),
            MBConv(48, 96, kernel_size=3, expand_ratio=4, stride=2),
            MBConv(96, 96, kernel_size=3, expand_ratio=4, stride=1),
            MBConv(96, 192, kernel_size=3, expand_ratio=4, stride=2),
            MBConv(192, 192, kernel_size=3, expand_ratio=4, stride=1),
            MBConv(192, 256, kernel_size=3, expand_ratio=4, stride=1),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.3)
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("EfficientNet-Lite: C(3x3,s2) → BN → Swish → MBConv×8 → GAP → D → FC")
print(f"파라미터: {sum(p.numel() for p in EfficientNetLite().parameters()):,}")



### 4.5 EfficientNet-B0 (Tan & Le, 2019) - 최고 성능 예상

**원본 논문의 B0 아키텍처** (입력: 224×224, 여기서는 128×128 적용)

| Stage | Operator | Resolution | Channels | Layers |
|-------|----------|------------|----------|--------|
| 1 | MBConv1, k3×3 | 112×112 | 16 | 1 |
| 2 | MBConv6, k3×3 | 56×56 | 24 | 2 |
| 3 | MBConv6, k5×5 | 28×28 | 40 | 2 |
| 4 | MBConv6, k3×3 | 14×14 | 80 | 3 |
| 5 | MBConv6, k5×5 | 14×14 | 112 | 3 |
| 6 | MBConv6, k5×5 | 7×7 | 192 | 4 |
| 7 | MBConv6, k3×3 | 7×7 | 320 | 1 |

**구조 특징**:
- **MBConv1**: expand_ratio=1 (채널 확장 없음)
- **MBConv6**: expand_ratio=6 (채널 6배 확장 후 축소)
- **k3×3 / k5×5**: Depthwise Conv 커널 크기
- **Compound Scaling**: 깊이, 너비, 해상도 균형있게 스케일링

**효과**:
- ✅ 원본 논문의 검증된 아키텍처
- ✅ 5×5 커널로 더 넓은 수용 영역
- ✅ 단계적 채널 증가로 세밀한 특징 학습
- ✅ ImageNet에서 검증된 구조

In [ ]:
class EfficientNetB0(nn.Module):
    """
    EfficientNet-B0 (Tan & Le, 2019) - 원본 논문 아키텍처

    입력: 128×128 RGB (원본은 224×224)
    """
    def __init__(self, num_classes=2):
        super().__init__()
        
        # B0 Configuration: (expand_ratio, channels, layers, stride, kernel_size)
        b0_config = [
            (1, 16, 1, 1, 3),
            (6, 24, 2, 2, 3),
            (6, 40, 2, 2, 5),
            (6, 80, 3, 2, 3),
            (6, 112, 3, 1, 5),
            (6, 192, 4, 2, 5),
            (6, 320, 1, 1, 3),
        ]
        
        layers = []
        # Stem: Conv3×3, stride 2, 32ch -> 128→64
        layers.extend([
            nn.Conv2d(3, 32, 3, 2, 1, bias=False),
            nn.BatchNorm2d(32),
            Swish()
        ])
        
        in_ch = 32
        for expand_ratio, out_ch, n_layers, stride, kernel_size in b0_config:
            for i in range(n_layers):
                s = stride if i == 0 else 1
                layers.append(
                    MBConv(in_ch, out_ch, kernel_size=kernel_size, 
                           expand_ratio=expand_ratio, stride=s, use_se=True)
                )
                in_ch = out_ch
        
        layers.extend([
            nn.Conv2d(320, 1280, 1, bias=False),
            nn.BatchNorm2d(1280),
            Swish(),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.2)
        ])
        
        self.layers = nn.ModuleList(layers)
        self.fc = nn.Linear(1280, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("EfficientNet-B0: Stem → MBConv×16 → Conv1×1(1280) → GAP → D → FC")
print(f"파라미터: {sum(p.numel() for p in EfficientNetB0().parameters()):,}")



## 5. 학습 함수

In [ ]:
# --------------------------------------------
# 학습/평가 유틸리티 함수 (H100 최적화)
# --------------------------------------------
scaler = GradScaler('cuda', enabled=USE_AMP)

def train_epoch(model, loader, criterion, optimizer):
    """Mixed Precision 학습 (H100 최적화)"""
    model.train()
    for images, labels in loader:
        images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)  # 메모리 효율적
        
        with autocast('cuda', enabled=USE_AMP):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

def evaluate(model, loader):
    """평가 함수 - Accuracy, F1 (Micro), F1 (Macro) 반환"""
    model.eval()
    preds, labels_list = [], []
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device, non_blocking=True)
            with autocast('cuda', enabled=USE_AMP):
                outputs = model(images)
            preds.extend(outputs.argmax(1).cpu().numpy())
            labels_list.extend(labels.numpy())
    
    acc = accuracy_score(labels_list, preds)
    f1_micro = f1_score(labels_list, preds, average='micro')
    f1_macro = f1_score(labels_list, preds, average='macro')
    return acc, f1_micro, f1_macro

def train_model(model, train_loader, val_loader, epochs=20, lr=0.001):
    """H100 최적화 학습 함수"""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    best_acc = 0
    best_state = None
    patience = 0
    
    for epoch in range(epochs):
        train_epoch(model, train_loader, criterion, optimizer)
        val_acc, _, _ = evaluate(model, val_loader)
        scheduler.step()
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            patience = 0
        else:
            patience += 1
            if patience >= 7:  # H100에서는 여유 있게
                break
    
    if best_state:
        model.load_state_dict(best_state)
    return model

print("H100 최적화 학습 함수 정의 완료")
print("  - Mixed Precision (autocast + GradScaler)")
print("  - F1 Score: Micro (=Accuracy) + Macro (클래스별 평균)")



## 6. StratifiedKFold 5겹 교차검증

In [ ]:
# --------------------------------------------
# 인덱스 기반 부분집합을 transform과 함께 래핑하는 헬퍼
# --------------------------------------------
class TransformDataset(Dataset):
    def __init__(self, base_dataset, indices, transform):
        self.base = base_dataset
        self.indices = indices
        self.transform = transform
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        i = self.indices[idx]
        image = Image.open(self.base.image_paths[i]).convert('RGB')
        return self.transform(image), self.base.labels[i]



In [ ]:
# --------------------------------------------
# StratifiedKFold 5겹 교차검증 실행 함수
# - 각 fold별 train/val 분리 후 테스트 세트는 고정 평가
# - torch.compile로 추가 최적화 시도
# --------------------------------------------
def run_kfold(model_class, model_name, n_splits=5):
    """H100 최적화 5겹 교차검증"""
    results = {
        'fold_acc': [], 'fold_f1_micro': [], 'fold_f1_macro': [],
        'test_acc': [], 'test_f1_micro': [], 'test_f1_macro': []
    }
    
    print()  # 구분을 위한 빈 줄
    print('='*60)
    print(model_name)
    print('='*60)
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    all_indices = np.arange(len(train_dataset))
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY
    )
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(all_indices, train_dataset.labels)):
        set_seed(42 + fold)
        print()
        print(f"Fold {fold+1}/{n_splits}")
        print(f"  Train: {len(train_idx)}, Val: {len(val_idx)}")
        
        train_subset = TransformDataset(train_dataset, train_idx, train_transform)
        val_subset = TransformDataset(train_dataset, val_idx, test_transform)
        
        train_loader = DataLoader(
            train_subset, 
            batch_size=BATCH_SIZE, 
            shuffle=True,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY,
            drop_last=True
        )
        val_loader = DataLoader(
            val_subset, 
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            pin_memory=PIN_MEMORY
        )
        
        model = model_class().to(device)
        
        if hasattr(torch, 'compile'):
            try:
                model = torch.compile(model, mode='reduce-overhead')
            except:
                pass
        
        model = train_model(model, train_loader, val_loader, epochs=30)
        
        fold_acc, fold_f1_micro, fold_f1_macro = evaluate(model, val_loader)
        test_acc, test_f1_micro, test_f1_macro = evaluate(model, test_loader)
        
        results['fold_acc'].append(fold_acc)
        results['fold_f1_micro'].append(fold_f1_micro)
        results['fold_f1_macro'].append(fold_f1_macro)
        results['test_acc'].append(test_acc)
        results['test_f1_micro'].append(test_f1_micro)
        results['test_f1_macro'].append(test_f1_macro)
        
        print(f"  Val  - Acc: {fold_acc:.4f}, F1(Micro): {fold_f1_micro:.4f}, F1(Macro): {fold_f1_macro:.4f}")
        print(f"  Test - Acc: {test_acc:.4f}, F1(Micro): {test_f1_micro:.4f}, F1(Macro): {test_f1_macro:.4f}")
        
        del model
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    return results



In [ ]:
lenet_results = run_kfold(LeNet5, "LeNet-5 (베이스라인)")

In [ ]:
vgg_results = run_kfold(VGGNet, "VGGNet-style")

In [ ]:
lite_results = run_kfold(EfficientNetLite, "EfficientNet-Lite")

In [ ]:
resnet_results = run_kfold(ResNetCNN, "ResNetCNN")

In [ ]:
b0_results = run_kfold(EfficientNetB0, "EfficientNet-B0 (최고 성능)")

## 7. 결과 분석

In [ ]:
def print_kfold_summary(results, name):
    # fold별 테스트 성능을 표 형식으로 출력
    print()
    print(name)
    print("-"*70)
    print(f"{'Fold':<6} {'Test Acc':<12} {'F1(Micro)':<12} {'F1(Macro)':<12}")
    for i in range(5):
        print(f"{i+1:<6} {results['test_acc'][i]:<12.4f} {results['test_f1_micro'][i]:<12.4f} {results['test_f1_macro'][i]:<12.4f}")
    print("-"*70)
    acc_mean, acc_std = np.mean(results['test_acc']), np.std(results['test_acc'])
    f1_micro_mean, f1_micro_std = np.mean(results['test_f1_micro']), np.std(results['test_f1_micro'])
    f1_macro_mean, f1_macro_std = np.mean(results['test_f1_macro']), np.std(results['test_f1_macro'])
    print(f"Mean   {acc_mean:.4f}±{acc_std:.4f}  {f1_micro_mean:.4f}±{f1_micro_std:.4f}  {f1_macro_mean:.4f}±{f1_macro_std:.4f}")
    return acc_mean, acc_std, f1_micro_mean, f1_micro_std, f1_macro_mean, f1_macro_std

print("="*60)
print("Final Results (Test Data)")
print("="*60)
lenet_stats = print_kfold_summary(lenet_results, "LeNet-5 (Baseline)")
vgg_stats = print_kfold_summary(vgg_results, "VGGNet-style")
resnet_stats = print_kfold_summary(resnet_results, "ResNetCNN")
lite_stats = print_kfold_summary(lite_results, "EfficientNet-Lite")
b0_stats = print_kfold_summary(b0_results, "EfficientNet-B0 (Best)")



In [ ]:
# --------------------------------------------
# 시각화 - 모델별 성능 비교 및 파라미터 수 출력
# --------------------------------------------
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

models = ['LeNet-5', 'VGGNet', 'ResNetCNN', 'Eff-Lite', 'Eff-B0']
all_results = [lenet_results, vgg_results, resnet_results, lite_results, b0_results]
colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#ff99ff']

# Accuracy
acc_means = [np.mean(r['test_acc']) for r in all_results]
acc_stds = [np.std(r['test_acc']) for r in all_results]
axes[0].bar(models, acc_means, yerr=acc_stds, capsize=5, color=colors)
axes[0].set_title('Test Accuracy')
axes[0].set_ylim([0.5, 1.0])
axes[0].set_ylabel('Score')

# F1 (Macro)
f1_macro_means = [np.mean(r['test_f1_macro']) for r in all_results]
f1_macro_stds = [np.std(r['test_f1_macro']) for r in all_results]
axes[1].bar(models, f1_macro_means, yerr=f1_macro_stds, capsize=5, color=colors)
axes[1].set_title('Test F1 Score (Macro)')
axes[1].set_ylim([0.5, 1.0])
axes[1].set_ylabel('Score')

plt.suptitle('Chihuahua vs Muffin - Model Performance Comparison', fontsize=14)
plt.tight_layout()
plt.show()

# Fold별 F1 (Macro) 성능 비교
fig, ax = plt.subplots(figsize=(14, 5))
x = np.arange(5)
width = 0.15

ax.bar(x - 2*width, lenet_results['test_f1_macro'], width, label='LeNet-5', color='#ff9999')
ax.bar(x - width, vgg_results['test_f1_macro'], width, label='VGGNet', color='#66b3ff')
ax.bar(x, resnet_results['test_f1_macro'], width, label='ResNetCNN', color='#99ff99')
ax.bar(x + width, lite_results['test_f1_macro'], width, label='Eff-Lite', color='#ffcc99')
ax.bar(x + 2*width, b0_results['test_f1_macro'], width, label='Eff-B0', color='#ff99ff')

ax.set_xlabel('Fold')
ax.set_ylabel('Test F1 (Macro)')
ax.set_title('Chihuahua vs Muffin - F1 (Macro) by Fold')
ax.set_xticks(x)
ax.set_xticklabels([f'{i+1}' for i in x])
ax.legend(loc='lower right')
ax.set_ylim([0.5, 1.0])
plt.tight_layout()
plt.show()

# 파라미터 수 비교
print()
print("="*60)
print("Model Parameters")
print("="*60)
models_params = [
    ("LeNet-5", LeNet5()),
    ("VGGNet", VGGNet()),
    ("ResNetCNN", ResNetCNN()),
    ("EfficientNet-Lite", EfficientNetLite()),
    ("EfficientNet-B0", EfficientNetB0())
]
for name, model in models_params:
    params = sum(p.numel() for p in model.parameters())
    print(f"{name:20s}: {params:>10,} params")



## 8. 결론

### 모델별 성능 비교

| 모델 | 구조 | 파라미터 | Accuracy | F1 (Micro) | F1 (Macro) |
|------|------|---------|----------|------------|------------|
| LeNet-5 (베이스라인) | C(5×5)→T→AvgP ×3 | ~560K | - | - | - |
| VGGNet-style | [C(3×3)→BN→R]×n→MaxP | ~1.9M | - | - | - |
| ResNetCNN | ResBlock + Skip | ~600K | - | - | - |
| EfficientNet-Lite | MBConv(k3) + SE | ~1.2M | - | - | - |
| EfficientNet-B0 (최고) | MBConv(k3,k5) + SE | ~4.0M | - | - | - |

### 평가 척도 해석
- **F1 (Micro)**: 전체 샘플에 대해 TP/FP/FN을 합산하여 계산. 균형 데이터셋에서는 Accuracy와 동일.
- **F1 (Macro)**: 각 클래스별 F1을 계산 후 평균. 클래스 불균형 시 소수 클래스의 성능을 더 잘 반영.

### 모델 분석

**1. LeNet-5 (베이스라인)**
- 수업시간에 배운 고전적 CNN 구조
- 5×5 커널과 Tanh 활성화 사용
- 얕은 구조로 치와와 눈과 머핀 초콜릿칩의 세밀한 차이 구분 한계

**2. VGGNet-style**
- 작은 3×3 커널을 깊게 쌓아 수용 영역 확보
- BN + ReLU로 학습 안정화
- LeNet-5보다 더 세밀한 특징 추출 가능

**3. ResNetCNN**
- Residual Connection으로 gradient flow 개선
- 더 깊은 네트워크 학습 가능
- Skip connection이 세밀한 특징 보존

**4. EfficientNet-Lite**
- 간소화된 MBConv 구조 (3×3 커널만 사용)
- SE Block으로 채널 attention
- 원본 B0보다 가볍고 빠른 학습

**5. EfficientNet-B0 (최고 성능 예상)**
- 원본 논문의 검증된 아키텍처
- 3×3과 5×5 커널 혼합 사용 → 다양한 수용 영역
- MBConv6 (expand_ratio=6)로 충분한 표현력
- 16개 MBConv 블록 + 1280ch Head
- ImageNet에서 검증된 구조

### EfficientNet-Lite vs B0 비교

| 비교 항목 | EfficientNet-Lite | EfficientNet-B0 |
|----------|------------------|-----------------|
| MBConv 블록 수 | 8개 | 16개 |
| 커널 크기 | 3×3만 | 3×3 + 5×5 혼합 |
| expand_ratio | 1~4 | 1~6 |
| Head 채널 | 256 | 1280 |
| 파라미터 | ~1.2M | ~4.0M |
| 예상 성능 | 중상 | 최고 |

### 결론
치와와와 머핀은 둥근 형태, 갈색 톤, 점 패턴이 유사하여 세밀한 특징 추출이 핵심.

**EfficientNet-B0**가 가장 높은 성능을 보일 것으로 예상:
1. 5×5 커널로 더 넓은 수용 영역 확보
2. expand_ratio=6으로 충분한 채널 확장
3. 16개 MBConv 블록으로 깊은 특징 학습
4. SE Block이 눈, 코, 입 등 치와와 고유의 특징에 attention